# Dimensionsreduktion mit Principal Component Analysis und t-SNE

## Laden der Bibliotheken

In [ ]:
import numpy as np
import pandas as pd
import keras.datasets.mnist
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

## PCA in zwei Dimensionen

Damit dir das Verfahren vollständig visualisieren können, betrachten wir zuerst ein sehr einfaches Beispiel in zwei Dimensionen. Dazu erzeugen wir eine Punktewolke

In [ ]:
n = 100
slope = np.random.uniform(-2, 2)
x = np.random.uniform(size=100)
y = x * slope + np.random.uniform(size=100)
xy = np.array([x, y]).T
xy -= xy.mean(axis=0)

maxval = abs(xy).max() * 1.05
plt.figure(figsize=[8, 8])
plt.scatter(xy[:, 0], xy[:, 1])
plt.xlim(-maxval, maxval)
plt.ylim(-maxval, maxval)
plt.show()

Nun führen wir die Dimensionsreduktion mit PCA durch

In [ ]:
pca = PCA(2)
pca.fit(xy)

Wir visualisieren nun die dimensionreduzierten Punkte gegenüber den ursprünglichen Punkten.

In [ ]:
def plot_reduced(dim):
    xy_reduced = np.matmul(pca.transform(xy)[:, :dim], pca.components_[:dim])
    plt.figure(figsize=[8, 8])
    plt.xlim(-maxval, maxval)
    plt.ylim(-maxval, maxval)
    plt.scatter(xy[:, 0], xy[:, 1])
    plt.scatter(xy_reduced[:, 0], xy_reduced[:, 1], color='red')
    var_total = (xy * xy).sum()
    var_reduced = (xy_reduced * xy_reduced).sum()
    plt.title(f'Percent of variance explained {var_reduced / var_total * 100:.1f}%')
    plt.show()

_ = interact(
    plot_reduced,
    dim=widgets.SelectionSlider(
        options=range(0, 3),
        description='Number of dimensions',
        layout=layout,
        style=style,
        orientation='horizontal',
    )
)


## PCA in 784 Dimensionen

Das folgende Beispiel ist deutlich praxisrelevanter. Man muss sich allerdings etwas in das Vorgehen eindenken, da die Anzahl der Dimensionen weit über der menschlichen Fähigkeit zur Visualisierung liegt. Wir betrachten im Folgenden Pixelbilder mit 28x28 = 784 Pixeln. Für jeden Punkt liegt ein Grauwert zwischen 0 und 1 vor (0 ist schwarz, 1 ist weiß). Effektiv entsprechend diese Graubilder also 784-dimensionalen Vektoren, für die wir (mit entsprechender Abstraktion) genau so vorgehen können wie im vorhergehenden Beispiel.

## Einlesen der Bilder

Zuerst lesen wir die Bilder ein. Es handelt sich um 60.000 Bilder von handschriftlichen geschriebenen Ziffern.

In [ ]:
(images, labels), _ = keras.datasets.mnist.load_data()
images = 1 - (images / 255)

Wir verifizieren die Dimensionen der Daten. Wir sehen, dass es sich tatsächlich um 60.000 Datensätze der Dimension 28 x 28 handelt.

In [ ]:
images.shape

Wir visualisieren nun beispielhaft einige der Bilder. Dabei ist i ist die Anzahl des Bildes und darf zwischen 0 und 59.999 liegen. Experimentieren Sie mit verschiedenen Werten für i. Unter dem Bild wird angegeben, welche Ziffer durch das Bild dargestellt wird.

In [ ]:
i = 4
plt.imshow(images[i], cmap='gray')
plt.show()
print(f'Ziffer: {labels[i]}')

Wir geben nun noch gesammelt die ersten 24 Bilder aus.

In [ ]:
plt.subplots(4, 5, figsize=(8, 6))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.axis('off')

## Dimensionsreduktion mit PCA

Nun führen wir für die Bilder ebenfalls eine Dimensionreduktion durch. Wir beschränken uns dabei auf Bilder der Ziffern 0 und 1, damit die Ergebnisse übersichtlicher sind.

In [ ]:
# Sie können statt 0 und 1 auch anderen Kombinationen ausprobieren, etwa [5, 6, 8] für die Ziffern 5, 6 und 8
selection = np.isin(labels, [0, 1])
n = selection.sum()
images_sel = images[selection].reshape(n, 28 * 28)
labels_sel = labels[selection]

Wir visualisieren die ersten 24 Bilder aus dem eingeschränkten Datensatz. Tatsächlich sind nur Nullen und Einsen zu sehen.

In [ ]:
plt.subplots(4, 6, figsize=(8, 6))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.imshow(images_sel[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

### Fitting der PCA

Nun führen wir die PCA auf unseren Bildern durch. Wieder werden die wichtigsten Dimensionen errechnet. Diese sind nun jedoch deutlich abstrakter und nehmen die Form von "prototypischen" Bildern an, aus denen die ursprünglichen Bilder zusammengesetzt werden.

In [ ]:
pca = PCA(28 * 28)
pca.fit(images_sel)

Wie zuvor visualisieren wir nun den Effekt der Dimensionsreduktion. Links sind die dimensionsreduzierten Bilder zu sehen, rechts zur Referenz die Originalbilder.

In [ ]:
def plot_reduced(dim):
    images_reconstructed = pca.mean_ + np.matmul(pca.transform(images_sel)[:, :dim], pca.components_[:dim])
    _, ax = plt.subplots(4, 10, figsize=(18, 6))
    plt.suptitle(f'Percent of variance explained {pca.explained_variance_ratio_[:dim].sum() * 100:.1f}%')
    for i in range(20):
        plt.subplot(4, 11, i + 6 * (i // 5) + 1)
        plt.imshow(images_reconstructed[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
        plt.subplot(4, 11, i + 6 * (i // 5) + 7)
        plt.imshow(images_sel[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()

i = interact(
    plot_reduced,
    dim=widgets.SelectionSlider(
        options=list(range(11)) + [20, 50, 75, 100, 200, 400, 784],
        description='Number of dimensions',
        continuous_update=False,
        style=style,
        layout=layout,
        orientation='horizontal',
    )
)

Zum Abschluss schauen wir unter die Haube. Zuerst betrachten wir, wie das Durchschnittsbild aussieht. Dieses kennen wir schon als das Ergebnis für die Reduktion auf 0 Dimensionen.

In [ ]:
plt.imshow(pca.mean_.reshape(28, 28), cmap='gray')

Und dann betrachten wir die ersten 5 Hauptkomponenten. Diese sind am besten als Differenzbilder zu verstehen, mit denen das Durchschnittsbild Stück für Stück verfeinert wird.

In [ ]:
plt.subplots(1, 5, figsize=(10, 2))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(pca.components_[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

## Dimensionsreduktion mit t-SNE

Nun widmen wir uns einem weiteren Verfahren, dem t-distributed Stochastic Neighbor Embedding (t-SNE). Wohingehen die PCA ein lineares Verfahren ist, kann t-SNE hochdimensionale Strukturen deutlich flexibler abbilden. Es dient jedoch vor allem zur menschlichen Visualisierung, eine direkte Rekonstruktion der Daten wie bei der PCA ist nicht möglich. Wie zuvor beschränken wir uns auf die Ziffern 0 und 1.

In [ ]:
# Auch hier können Sie wieder mit verschiedenen Kombinationen von Ziffern experimentieren.
selection = np.isin(labels, [0, 1])
n = selection.sum()
images_sel = images[selection].reshape(n, 28 * 28)
labels_sel = labels[selection]

Die Laufzeit von t-SNE ist etwas länger

In [ ]:
%%time

num_samples = 2000

# try different values for perplexity and also n_iter
tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500)
tsne_result = tsne.fit_transform(images_sel[:num_samples])

### Darstellung des Embeddings

In [ ]:
plot_frame = pd.DataFrame(tsne_result).assign(label=labels_sel[:num_samples])
plt.figure(figsize=(12, 12))
for l, g in plot_frame.groupby('label'):
    plt.scatter(g[0], g[1], label=l)
plt.legend()
plt.show()

### Blick auf die einzelnen Samples

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

plt.figure(figsize=(12, 12))
plt.xlim(-10, 10)
plt.ylim(-10, 10)
tnse_result_rescaled = tsne_result / np.abs(tsne_result).max() * 9.5
for i in range(300):
    pixels = images_sel[i].copy()
    pixels[pixels > 0.99] = np.nan
    im = OffsetImage(pixels.reshape(28, 28), cmap='gray')
    ab = AnnotationBbox(im, tnse_result_rescaled[i], frameon=False)
    plt.gca().add_artist(ab)

plt.show()